# Introduction
This notebook shows how to submit a job with current `Jupyter` notebook. To authenticated by `OpenPAI`, please store your system information in `openpai.json` (`OpenPAI` configuration file), including key information like **pai_uri**, **user**, **passwd** and **hdfs_web_uri**

Note:
- **pai_uri** is in the format of `http://x.x.x.x`
- **hdfs_web_uri** is in the format of `http://x.x.x.x:port`
- <red>***Keep your configuration secret. Never commit it***</red>

# Prepare environment

In [ ]:
import importlib
import os, sys

sdk_module_name = 'paiclient'
module_spec = importlib.util.find_spec(sdk_module_name)

if module_spec is None:
    ! pip install -U https://codeload.github.com/mydmdm/paiclient/zip/master

local_sdk_path = '..' # set it if you want to debug the sdk (client) itself

if local_sdk_path and os.path.isdir(os.path.join(local_sdk_path, sdk_module_name)) and local_sdk_path not in sys.path:
    sys.path.insert(0, local_sdk_path)
    %load_ext autoreload
    %autoreload 2
    
print(importlib.util.find_spec(sdk_module_name))

# Do whatever you want

In [ ]:
from paiclient.core import in_job_container, in_debug_mode


if in_job_container():
    print('I am in a container')
else:
    print('I am out of a container')

# Submit to `OpenPAI`
After debugging in your environment, just submit current notebook and let it run.

In [ ]:
from paiclient.notebook import parse_notebook_path, submit_notebook
from paiclient.core import Client
from uuid import uuid4

if not in_job_container() or in_debug_mode():
    client = Client.from_json('openpai.json')
    _, name, _ = parse_notebook_path()
    job_name = '{}_{}'.format(name, uuid4().hex)
    job_dir = '/user/{}/jobs/{}'.format(client.user, job_name)
    submit_notebook(
        client,
        image='jupyter/minimal-notebook', # 'ubuntu:16.04'
        job_dir=job_dir
    )
    print(client.get_job_link(job_name))
else:
    client = Client.from_envs()
    print(vars(client))